In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn

In [2]:
folder_path = '/content/drive/MyDrive/python_data/kaggle/airbnb/data/'
files = ['age_gender_bkts.csv', 'countries.csv', 'sessions.csv', 'test_users.csv', 'train_users_2.csv']

In [3]:
train_df = pd.read_csv(folder_path+files[4])
test_df = pd.read_csv(folder_path+files[3])
sessions = pd.read_csv(folder_path+files[2])
countries = pd.read_csv(folder_path+files[1])
age_gender = pd.read_csv(folder_path+files[0])

# EDA

In [4]:
train_df.head(3)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US


## Session

In [5]:
sessions.head(3)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0


## Country

In [6]:
countries

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06
5,GB,54.633220,-3.432277,6883.6590,243610.0,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22
8,PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45
9,US,36.966427,-95.844030,0.0000,9826675.0,eng,0.00


## 確認target 是否有data imbalance問題
1. 6成NDF，3成US
2. Evaluation metrics：NDCG
3. data imbalance 問題

In [7]:
print('目的地占比 %')
(train_df.iloc[:, -1].value_counts()/len(train_df))*100

目的地占比 %


NDF      58.347349
US       29.222632
other     4.728954
FR        2.353233
IT        1.328174
GB        1.088774
ES        1.053638
CA        0.669006
DE        0.497070
NL        0.356991
AU        0.252517
PT        0.101663
Name: country_destination, dtype: float64

In [8]:
print(len(sessions))
sessions.head(3)

10567737


,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0


In [9]:
# new_session = pd.merge(sessions, train_df, left_on='user_id', right_on= 'id')[['user_id', 'action', 'action_type', 'action_detail', 'device_type', 'secs_elapsed', 'country_destination']]

In [10]:
# new_session.groupby(['action_detail'])['country_destination'].value_counts()
# pd.get_dummies(sessions)
print(sessions['action'].nunique())
print(sessions['action_type'].nunique())
print(sessions['action_detail'].nunique())

359
10
155


In [11]:
# sessions['action_detail'].apply(lambda x: x.split('_')[0])
clean_session = sessions[['user_id', 'action_detail']].dropna().drop_duplicates()
user_action_feature = pd.get_dummies(clean_session, columns= ['action_detail']).groupby(['user_id']).max()

In [12]:
# new_session.groupby(['action_detail'])['country_destination'].apply(lambda x: x.value_counts().head(3)).tail(20)

## date_first_booking 缺值 -> NDF, 沒缺 -> 沒有NDF

In [13]:
train_df[train_df['date_first_booking'].isnull()]['country_destination'].value_counts()

NDF    124543
Name: country_destination, dtype: int64

## Age 補值？

In [14]:
print(round(len(train_df[train_df['age'].isnull()])*100/len(train_df), 2), '% of train data do not have age value')

41.22 % of train data do not have age value


In [15]:
# train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       213451 non-null  object 
 1   date_account_created     213451 non-null  object 
 2   timestamp_first_active   213451 non-null  int64  
 3   date_first_booking       88908 non-null   object 
 4   gender                   213451 non-null  object 
 5   age                      125461 non-null  float64
 6   signup_method            213451 non-null  object 
 7   signup_flow              213451 non-null  int64  
 8   language                 213451 non-null  object 
 9   affiliate_channel        213451 non-null  object 
 10  affiliate_provider       213451 non-null  object 
 11  first_affiliate_tracked  207386 non-null  object 
 12  signup_app               213451 non-null  object 
 13  first_device_type        213451 non-null  object 
 14  firs

## Test data

In [16]:
test_df.head(3)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome


In [17]:
# test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62096 entries, 0 to 62095
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       62096 non-null  object 
 1   date_account_created     62096 non-null  object 
 2   timestamp_first_active   62096 non-null  int64  
 3   date_first_booking       0 non-null      float64
 4   gender                   62096 non-null  object 
 5   age                      33220 non-null  float64
 6   signup_method            62096 non-null  object 
 7   signup_flow              62096 non-null  int64  
 8   language                 62096 non-null  object 
 9   affiliate_channel        62096 non-null  object 
 10  affiliate_provider       62096 non-null  object 
 11  first_affiliate_tracked  62076 non-null  object 
 12  signup_app               62096 non-null  object 
 13  first_device_type        62096 non-null  object 
 14  first_browser         

In [18]:
test_df['date_first_booking'].value_counts()

Series([], Name: date_first_booking, dtype: int64)

In [19]:
print(round(len(test_df[test_df['age'].isnull()])*100/len(test_df), 2), '% of the test data do not have age')

46.5 % of the test data do not have age


# Preprocessing

In [20]:
train_df.head(3)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US


In [24]:
train_df['year_account_created'] = train_df['date_account_created'].apply(lambda x: x.split('-')[0])
train_df['month_account_created'] = train_df['date_account_created'].apply(lambda x: x.split('-')[1])
train_df_drop = train_df.drop(columns= ['id', 'date_first_booking', 'date_account_created', 'timestamp_first_active', 'age', 'country_destination'], axis=0)
train_df_drop['signup_flow'] = train_df_drop['signup_flow'].astype(str)
train_df_drop.head(3)

,id,gender,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,year_account_created,month_account_created
0,gxn3p5htnn,-unknown-,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,2010,06
1,820tgsjxq7,MALE,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,2011,05
2,4ft3gnwmtx,FEMALE,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,2010,09


In [27]:
dummy_feature = pd.get_dummies(train_df_drop)
booking_feature = pd.concat([train_df[['id']], dummy_feature], axis= 1)
train_X = pd.merge(booking_feature, user_action_feature, left_on= 'id', right_on='user_id', how= 'left')

,id,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,signup_method_google,signup_flow_0,signup_flow_1,signup_flow_10,signup_flow_12,signup_flow_15,signup_flow_16,signup_flow_2,signup_flow_20,signup_flow_21,signup_flow_23,signup_flow_24,signup_flow_25,signup_flow_3,signup_flow_4,signup_flow_5,signup_flow_6,signup_flow_8,language_ca,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,language_hr,language_hu,language_id,language_is,language_it,language_ja,...,action_detail_signup_modal,action_detail_similar_listings,action_detail_special_offer_field,action_detail_terms_and_privacy,action_detail_toggle_archived_thread,action_detail_toggle_starred_thread,action_detail_tos_2014,action_detail_translate_listing_reviews,action_detail_translations,action_detail_trip_availability,action_detail_unavailable_dates,action_detail_update_listing,action_detail_update_listing_description,action_detail_update_user,action_detail_update_user_profile,action_detail_user_friend_recommendations,action_detail_user_languages,action_detail_user_listings,action_detail_user_profile,action_detail_user_profile_content_update,action_detail_user_reviews,action_detail_user_social_connections,action_detail_user_tax_forms,action_detail_user_wishlists,action_detail_view_ghosting_reasons,action_detail_view_ghostings,action_detail_view_identity_verifications,action_detail_view_listing,action_detail_view_locations,action_detail_view_reservations,action_detail_view_resolutions,action_detail_view_search_results,action_detail_view_security_checks,action_detail_view_user_real_names,action_detail_wishlist,action_detail_wishlist_content_update,action_detail_wishlist_note,action_detail_your_listings,action_detail_your_reservations,action_detail_your_trips
0,gxn3p5htnn,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,820tgsjxq7,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4ft3gnwmtx,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
train_y = train_df['country_destination']
len(train_X) == len(train_y)

True

In [39]:
print(f'有全部feature佔全部training資料比例: {round(len(train_X.dropna())*100/len(train_X), 2)}%')

有全部feature佔全部training資料比例: 34.58%


# Metric：NDCG

* predict multiclass with probability(n_class = 1+n_countries)
* loss function: cross entropy